In [405]:
import pandas as pd
import numpy as np
from pathlib import Path
import requests
from urllib.parse import urlencode
from yarl import URL

## Getting data from the AQS API

In [323]:
email = f'winston.degraw@gmail.com'
key = 'carmelswift48'

In [428]:
def get_state_codes(email, key):
    '''
    Queries AQS to get the list of state codes
    '''
    base_url = 'https://aqs.epa.gov/data/api/list/states'
    params = {
        'email' : email,
        'key'   : key  
    }
    
    url = URL('?'.join([base_url, urlencode(params)]), encoded=False)
    res = requests.get(url)

    if res.status_code == 200:
        posts = pd.DataFrame(res.json()['Data'])
        return posts
    else:
        print(f'status: {res.status_code}')
    return None

#### Only need to query the state list once, so storing it to a variable to loop over.

In [429]:
state_codes = get_state_codes(email,key)

In [433]:
for code, state in state_codes.values:
    print(code, state)

01 Alabama
02 Alaska
04 Arizona
05 Arkansas
06 California
08 Colorado
09 Connecticut
10 Delaware
11 District Of Columbia
12 Florida
13 Georgia
15 Hawaii
16 Idaho
17 Illinois
18 Indiana
19 Iowa
20 Kansas
21 Kentucky
22 Louisiana
23 Maine
24 Maryland
25 Massachusetts
26 Michigan
27 Minnesota
28 Mississippi
29 Missouri
30 Montana
31 Nebraska
32 Nevada
33 New Hampshire
34 New Jersey
35 New Mexico
36 New York
37 North Carolina
38 North Dakota
39 Ohio
40 Oklahoma
41 Oregon
42 Pennsylvania
44 Rhode Island
45 South Carolina
46 South Dakota
47 Tennessee
48 Texas
49 Utah
50 Vermont
51 Virginia
53 Washington
54 West Virginia
55 Wisconsin
56 Wyoming
66 Guam
72 Puerto Rico
78 Virgin Islands
80 Country Of Mexico
CC Canada


In [339]:
def get_county_codes(email, key, state):
    '''
    Queries AQS to get the list of county codes for the given state
    '''
    base_url = 'https://aqs.epa.gov/data/api/list/countiesByState'
    params = {
        'email' : email,
        'key'   : key  ,
        'state' : state
    }
    
    url = URL('?'.join([base_url, urlencode(params)]), encoded=False)
    res = requests.get(url)

    if res.status_code == 200:
        posts = pd.DataFrame(res.json()['Data'])
        return posts['code']
    else:
        print(f'status: {res.status_code}')
    return None

In [376]:
def get_param_class_codes(email, key):
    '''
    Queries AQS to get the codes representing the classes of measured air pollutants
    '''
    base_url = 'https://aqs.epa.gov/data/api/list/classes'
    params = {
        'email' : email,
        'key'   : key  
    }
    
    url = URL('?'.join([base_url, urlencode(params)]), encoded=False)
    res = requests.get(url)

    if res.status_code == 200:
        posts = pd.DataFrame(res.json()['Data'])
        return posts
    else:
        print(f'status: {res.status_code}')
    return None

In [ ]:
get_param_class_codes(email,key)

,code,value_represented
0,AIRNOW MAPS,The parameters represented on AirNow maps (881...
1,ALL,Select all Parameters Available
2,AQI POLLUTANTS,Pollutants that have an AQI Defined
3,CORE_HAPS,Urban Air Toxic Pollutants
4,CRITERIA,Criteria Pollutants
5,CSN DART,List of CSN speciation parameters to populate ...
6,FORECAST,Parameters routinely extracted by AirNow (STI)
7,HAPS,Hazardous Air Pollutants
8,IMPROVE CARBON,IMPROVE Carbon Parameters
9,IMPROVE_SPECIATION,PM2.5 Speciated Parameters Measured at IMPROVE...


#### The `CRITERIA` class seems to be the suggested parameter class, so going to use that (`AQI POLLUTANTS` contains basically the same info, but with some aggregate stats as well that we don't want)

In [399]:
def get_param_codes_by_class(email, key, pc):
    '''
    Queries AQS to get the codes representing the parameters in parameter class (pc)
    '''
    base_url = 'https://aqs.epa.gov/data/api/list/parametersByClass'
    params = {
        'email' : email,
        'key'   : key  ,
        'pc'    : pc
    }
    
    url = URL('?'.join([base_url, urlencode(params)]), encoded=False)
    res = requests.get(url)

    if res.status_code == 200:
        posts = pd.DataFrame(res.json()['Data'])
        return posts['code']
    else:
        print(f'status: {res.status_code}')
    return None

#### Also only need to get this once, so storing it to a variable

In [400]:
param_codes = get_param_codes_by_class(email, key, 'CRITERIA')

In [353]:
def aqs_api_annual_county(email, key, param, bdate, edate, state, county):
    '''
    Query the AQS API to get the annual summary data for the given parameter, state, county
    bdate and edate have to be in the same year for the request to work
    '''
    base_url = 'https://aqs.epa.gov/data/api/annualData/byCounty'
    params = {
        'email' : email,
        'key'   : key  ,
        'param' : param,
        'bdate' : bdate,
        'edate' : edate,
        'state' : state,
        'county': county
    }
    
    url = URL('?'.join([base_url, urlencode(params)]), encoded=False)
    res = requests.get(url)
    
    if res.status_code == 200:
        posts = pd.DataFrame(res.json()['Data'])
        return posts
    else:
        print(f'status: {res.status_code}')
    return None

### Now going to try to loop over all states, for each state loop over each year, for each year loop over each parameter, and for each parameter loop over each county in the state to gather the data for that county. Will have to query AQS for each state to get the county codes.

In [420]:
# put in beginning year, end year + 1
years = np.arange(2005, 2006).astype(str)
years

array(['2005'], dtype='<U21')

In [421]:
data_list = []
for state in state_codes:
    # Query for county codes at the top to avoid repeated queries
    county_codes = get_county_codes(email,key,state)
    for year in years:
        print('--------------------')
        print(f'Collecting data for state {state} in year {year}')
        print('--------------------')
        for param in param_codes:
            for county in county_codes:
                data_list.append( aqs_api_annual_county(email, key, param, year+'0101', year+'1231', state, county) )

--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------


KeyboardInterrupt: 

In [427]:
data = pd.concat(data_list, ignore_index=True)
data

,state_code,county_code,site_number,parameter_code,poc,latitude,longitude,datum,parameter,sample_duration_code,...,fiftieth_percentile,tenth_percentile,local_site_name,site_address,state,county,city,cbsa_code,cbsa,date_of_last_change
0,01,073,6004,42101,1,33.565278,-86.796389,WGS84,Carbon monoxide,1,...,0.4,0.1,Sloss Shuttlesworth,4113 SHUTTLESWORTH DRIVE,Alabama,Jefferson,Birmingham,13820,"Birmingham-Hoover, AL",2021-11-08
1,01,073,6004,42101,1,33.565278,-86.796389,WGS84,Carbon monoxide,Z,...,0.4,0.1,Sloss Shuttlesworth,4113 SHUTTLESWORTH DRIVE,Alabama,Jefferson,Birmingham,13820,"Birmingham-Hoover, AL",2021-11-08
2,01,073,1003,42101,1,33.485556,-86.915000,WGS84,Carbon monoxide,1,...,0.6,0.3,Fairfield,"FAIRFIELD, PFD, 5229 COURT B",Alabama,Jefferson,Fairfield,13820,"Birmingham-Hoover, AL",2021-11-08
3,01,073,1003,42101,1,33.485556,-86.915000,WGS84,Carbon monoxide,Z,...,0.6,0.3,Fairfield,"FAIRFIELD, PFD, 5229 COURT B",Alabama,Jefferson,Fairfield,13820,"Birmingham-Hoover, AL",2021-11-08
4,01,073,0028,42101,1,33.529444,-86.850278,WGS84,Carbon monoxide,1,...,0.2,0.0,None,"EAST THOMAS, FINLEY, 841 FINLEY AVE. BP.",Alabama,Jefferson,Birmingham,13820,"Birmingham-Hoover, AL",2021-11-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3293,08,031,0002,42401,1,39.751184,-104.987625,WGS84,Sulfur dioxide,1,...,8.0,1.0,DENVER - CAMP,2105 BROADWAY,Colorado,Denver,Denver,19740,"Denver-Aurora-Lakewood, CO",2021-11-09
3294,08,031,0002,42401,1,39.751184,-104.987625,WGS84,Sulfur dioxide,X,...,1.6,0.2,DENVER - CAMP,2105 BROADWAY,Colorado,Denver,Denver,19740,"Denver-Aurora-Lakewood, CO",2021-11-09
3295,08,031,0002,42401,1,39.751184,-104.987625,WGS84,Sulfur dioxide,Y,...,1.0,0.0,DENVER - CAMP,2105 BROADWAY,Colorado,Denver,Denver,19740,"Denver-Aurora-Lakewood, CO",2021-11-09
3296,08,001,3001,42602,1,39.838119,-104.949840,WGS84,Nitrogen dioxide (NO2),1,...,17.0,1.0,Welby,3174 E. 78TH AVE.,Colorado,Adams,Welby,19740,"Denver-Aurora-Lakewood, CO",2021-11-09


In [425]:
data.to_csv('data/AQS_data_test.csv')

In [ ]:
lbw = pd.read_csv('data/Natality_by_year_2007-2021.txt', sep="\t")
lbw.columns = [col.lower().replace(' ', '_') for col in lbw.columns]

In [29]:
lbw.drop(columns='notes').dropna().sort_values(by='year', ascending=False)

,state,state_code,county,county_code,infant_birth_weight_12,infant_birth_weight_12_code,year,year_code,births,%_of_total_births,average_birth_weight,average_lmp_gestational_age,average_oe_gestational_age
21213,Wyoming,56.0,"Unidentified Counties, WY",56999.0,2000 - 2499 grams,5.0,2021.0,2021.0,230.0,0.01%,2303.27,36.19,36.30
11144,Missouri,29.0,"St. Charles County, MO",29183.0,2000 - 2499 grams,5.0,2021.0,2021.0,94.0,0.00%,2324.00,36.36,35.80
11120,Missouri,29.0,"St. Charles County, MO",29183.0,1000 - 1499 grams,3.0,2021.0,2021.0,20.0,0.00%,1277.05,30.30,30.05
1955,California,6.0,"San Diego County, CA",6073.0,1000 - 1499 grams,3.0,2021.0,2021.0,143.0,0.01%,1277.10,30.43,30.19
11108,Missouri,29.0,"St. Charles County, MO",29183.0,500 - 999 grams,2.0,2021.0,2021.0,10.0,0.00%,753.10,30.00,25.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7788,Iowa,19.0,"Unidentified Counties, IA",19999.0,500 - 999 grams,2.0,2007.0,2007.0,48.0,0.00%,785.85,26.13,25.72
15709,Ohio,39.0,"Warren County, OH",39165.0,1500 - 1999 grams,4.0,2007.0,2007.0,19.0,0.00%,1774.05,33.26,32.89
19108,Texas,48.0,"Travis County, TX",48453.0,500 - 999 grams,2.0,2007.0,2007.0,71.0,0.00%,759.96,25.44,25.31
15704,Ohio,39.0,"Warren County, OH",39165.0,1000 - 1499 grams,3.0,2007.0,2007.0,15.0,0.00%,1188.47,31.20,29.80


In [48]:
from pathlib import Path
data_path = Path('data/')
aqi_files = [file for file in data_path.glob('*aqi*') if file.is_file()]
aqi_files.sort()
aqi = pd.read_csv(aqi_files[0])
for file in aqi_files[1:]:
    aqi = pd.concat([aqi, pd.read_csv(file)])
aqi.columns = [col.lower().replace(' ', '_') for col in aqi.columns]
aqi.to_csv('data/aqi_by_year_2006-2021.csv', index=False)

In [49]:
aqi.sort_values(by='days_with_aqi').sample(50)

,state,county,year,days_with_aqi,good_days,moderate_days,unhealthy_for_sensitive_groups_days,unhealthy_days,very_unhealthy_days,hazardous_days,max_aqi,90th_percentile_aqi,median_aqi,days_co,days_no2,days_ozone,days_pm2.5,days_pm10
798,Pennsylvania,Northampton,2011,365,165,189,9,2,0,0,152,84,53,0,9,89,264,3
395,Maryland,Harford,2019,359,261,89,9,0,0,0,133,71,44,0,0,231,128,0
81,California,Napa,2010,365,293,70,1,1,0,0,159,60,39,0,3,184,178,0
853,South Carolina,Greenville,2006,365,118,240,5,2,0,0,159,82,59,0,16,0,348,1
279,Indiana,Henry,2016,119,106,13,0,0,0,0,65,51,28,0,0,0,119,0
775,Pennsylvania,Luzerne,2012,366,329,35,2,0,0,0,129,51,33,0,0,348,0,18
262,Illinois,McHenry,2009,365,242,123,0,0,0,0,100,67,43,0,0,174,191,0
98,California,Ventura,2021,365,208,147,10,0,0,0,122,78,48,0,0,278,75,12
402,Maine,Knox,2007,179,151,22,4,2,0,0,177,71,34,0,0,179,0,0
300,Iowa,Pottawattamie,2021,124,100,23,1,0,0,0,117,57,33,0,0,0,120,4


In [408]:
aqi['year'].value_counts()

2006    1091
2007    1082
2008    1077
2009    1077
2010    1076
2011    1071
2012    1050
2013    1044
2015    1042
2014    1036
2016    1030
2017    1029
2018    1021
2019    1020
2020    1003
2021    1002
Name: year, dtype: int64